In [172]:
# low-level modules
class RawSubtitle:
  """
  Converts a file with .vtt format to python object with some attrs.
  Also tokenizes the .vtt content
  """
  def __init__(self, subtitle_path: str = None, offset: int = 14):
    self.subtitle_path = subtitle_path
    self._lines = None
    self._breaks =  None
    self._set_lines(offset)
    self._set_breaks()

  def _set_lines(self, offset):

     _lines = open(self.subtitle_path).readlines()[offset:]
     _lines[0] = '\n'
     self._lines = _lines

  def _set_breaks(self):
      self._breaks = [i for i in range(len(self._lines)) if self._lines[i] == '\n']

  def tokenized(self) -> list:
    # The algorithm works by selecting the items between every 2 single line breaks
    return [ self._lines[self._breaks[i]:self._breaks[i+1]][1:] for i in range(len(self._breaks)-1) ]


def get_numbers(string: str) -> int:
  _number = ''.join([i for i in string if i.isnumeric()])
  return int(_number)


class Dialog:
  """
  A Dialog object is a representative of every dialog and
  encapsulates dialog data
  """
  def __init__(self, raw_dialog: list):
    self.number = get_numbers(raw_dialog[0])
    self.start = raw_dialog[1][:12]
    self.end = raw_dialog[1][17:29]
    self.metadata = raw_dialog[1][29:]
    self.text = raw_dialog[2:]

  def __repr__(self) -> str:
    return f'\n{self.number}\n {self.start} --> {self.end} {self.metadata}\n {"".join(self.text)}'


class DialogFactory:
  """
  Factory with list dialogs as the product
  """
  def __init__(self, subtitle_file_path: str) -> None:
    self.subtitle_file_path = subtitle_file_path
    self.dialogs = self.factory()

  def factory(self) -> list:
    _tokens = RawSubtitle(self.subtitle_file_path).tokenized()
    dialogs = [Dialog(token) for token in _tokens]
    return dialogs

  def find_unique_times(self) -> dict:
    time_dict = {}
    _reps = []
    for i in self.dialogs:
      if not i.start in list(time_dict.keys()):
        time_dict.update({i.start:i})
      elif i.start in list(time_dict.keys()):
        _reps.append(i)

    for i in _reps:
      time_dict[i.start].text += i.text
    return time_dict

  def write_to_vtt(self, file_name) -> None:
    with open(f'{file_name}.vtt', 'w') as f:
      for dialog in self.dialogs:
        f.write(f'\n{dialog}')

# renumbering: change the number attr in the dialogs
def renumber_dialogs(factory:DialogFactory):
  _times_dic = factory.find_unique_times()
  for i in _times_dic.values():
    l = list(_times_dic.values())
    i.number = l.index(i) + 1
  factory.dialogs = list(_times_dic.values())

In [174]:
#main module

# reset the start and end times in the de version
def reset_de_time(factory:DialogFactory):
  for i in range(len(de_dialogs.dialogs)):
    de_dialogs.dialogs[i].start = en_dialogs.dialogs[i].start
    de_dialogs.dialogs[i].end = en_dialogs.dialogs[i].end

if __name__ == '__main__':
  print('working...')
  de_dialogs = DialogFactory('de_70105212.vtt')
  en_dialogs = DialogFactory('en_70105212.vtt')
  renumber_dialogs(en_dialogs)
  reset_de_time(de_dialogs)
  en_dialogs.write_to_vtt('enEdited')
  de_dialogs.write_to_vtt('deEdited')

working...
